## Install Dependencies

In [1]:
sc.install_pypi_package('pip==22.0.4')
sc.install_pypi_package('fsspec')
sc.install_pypi_package('s3fs')
sc.install_pypi_package("pandas==0.25.1")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1646879473112_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/4d/16/0a14ca596f30316efd412a60bdfac02a7259bf8673d4d917dc60b9a21812/pip-22.0.4-py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1

  Cache entry deserialization failed, entry ignored

In [12]:
sc.list_packages()

VBox()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/1 with error payload: {"msg":"Session '1' not found."}


## Import Packages

In [5]:
import s3fs
import fsspec
import pandas as pd
from pyspark.sql import Window
from pyspark.sql import functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load Header Names

In [6]:
columns = (spark.read.text('s3://bdcc-2022-final-project/headers.csv')
           .toPandas()['value'].tolist()[1:])
columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_ADM2Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_FullName', 'ActionGeo_CountryCo

## Load Data

In [7]:
# main

# year = 2019
data = (spark.read.text('s3://gdelt-open-data/v2/events/*.csv')
        .select(F.split(F.lower('value'), '\t').alias('split')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Mean Average Tone

In [14]:
# change the keyword to filter the dataset and save average tone for that day

keyword = 'netflix'

((data
 .withColumn('DATEADDED', F.col('split').getItem(columns.index('DATEADDED')))
 .withColumn('Actor1Name', F.col('split').getItem(columns.index('Actor1Name')))
 .withColumn('Actor2Name', F.col('split').getItem(columns.index('Actor2Name')))
 .withColumn('AvgTone', F.col('split').getItem(columns.index('AvgTone')).cast('int'))
 .withColumn('SOURCEURL', F.col('split').getItem(columns.index('SOURCEURL')))
 .drop('split')
 .filter((F.col('Actor1Name').contains(keyword)) |
         (F.col('Actor2Name').contains(keyword)) |
         (F.col('SOURCEURL').contains(keyword)))
 .groupBy(F.substring('DATEADDED', 1, 8).alias('DATEADDED'))
 .agg(F.mean('AvgTone').alias('Daily Average Tone'))
 .orderBy('DATEADDED')
).toPandas()
 .to_csv('s3://bdcc-2022-final-project/{}.csv'.format(keyword), index=False))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cumulative Sum Average Tone

In [11]:
# change the keyword to filter the dataset and save cumulative tone for that day

keyword = 'microsoft'

windowval = (Window.partitionBy(F.substring('DATEADDED', 1, 8)).orderBy('DATEADDED')
             .rowsBetween(Window.unboundedPreceding, 0))

((data
  .withColumn('DATEADDED', F.col('split').getItem(columns.index('DATEADDED')))
  .withColumn('Actor1Name', F.col('split').getItem(columns.index('Actor1Name')))
  .withColumn('Actor2Name', F.col('split').getItem(columns.index('Actor2Name')))
  .withColumn('AvgTone', F.col('split').getItem(columns.index('AvgTone')).cast('int'))
  .withColumn('SOURCEURL', F.col('split').getItem(columns.index('SOURCEURL')))
  .drop('split')
  .filter((F.col('Actor1Name').contains(keyword)) |
          (F.col('Actor2Name').contains(keyword)) |
          (F.col('SOURCEURL').contains(keyword)))
  .withColumn('cum_sum', F.sum('AvgTone').over(windowval))
  .groupBy(F.substring('DATEADDED', 1, 8).alias('DATEADDED'))
  .agg(F.mean('cum_sum').alias('Daily Average Tone'))
  .orderBy('DATEADDED')
 ).toPandas()
  .to_csv('s3://bdcc-2022-final-project/{}_cumsum.csv'.format(keyword),
          index=False))

VBox()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/1 with error payload: {"msg":"Session '1' not found."}
